In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import pandas as pd

files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        f = os.path.join(dirname, filename)
        files.append(f)
        
#Question 1
total_likes = 0
total_entries = 0
# views = like + rt + r/0.5
total_rt = 0
total_replies = 0
total_views = 0

for file in files:
    data = pd.read_csv(file)
#     print(data.nlikes)
    df = pd.DataFrame(data)
    total_likes += df['nlikes'].sum(axis=0)
    total_replies += df['nreplies'].sum(axis=0)
    total_rt += df['nreplies'].sum(axis=0)
    total_entries += len(df.index)
    
# print(total_likes)
# print(total_entries)
print(total_likes/total_entries)
    
    

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Question 2
total_views = 0
total_entries = 0
for file in files:
    data = pd.read_csv(file)
    df = pd.DataFrame(data)
    likes = df['nlikes'].sum(axis=0)
    replies = df['nreplies'].sum(axis=0)
    rt = df['nretweets'].sum(axis=0)
    total_entries += len(df.index)
    total_views += (likes + replies + rt)/0.05
    
print(total_views)
avg_view = total_views / total_entries
print(total_views/total_entries)

In [3]:
#Question 3

from requests import get
from io import BytesIO
from PIL import Image

# get total image size
def parse_url(photo_urls):
    urls = photo_urls[1:-1].split(', ')
    urls = list(map(lambda x:x[1:-1], urls))
    return urls

# get total number of char's in the tweet
total_len_tweets = 0
total_tweets = 0
for file in files:
    data = pd.read_csv(file)
    df = pd.DataFrame(data)
    tweet = data["tweet"]
    total_tweets += len(df.index)
    for t in tweet:
        total_len_tweets += len(t)


photo_size_bytes = 0
sample_size = 0
total_num_photo = 0
for file in files:
    data = pd.read_csv(file)
    photos = data['photos']
#     print(photos)
    for i, photo in enumerate(photos):
        if photo == '[]': 
            continue

        photo_urls = parse_url(photo)
#         print(photo_urls)
        total_num_photo += len(photo_urls)

        # take some samples of photos to calculate the average photo size
        if i % 10 == 0: 
            for url in photo_urls:
#                 print(url)
                response = get(url)
                try:
                    img = Image.open(BytesIO(response.content))
                    width, height = img.size
                    # jpg images has 3 bytes / pixel and photos are all jpg
                    photo_size_bytes += width * height * 3 
                    sample_size += 1
                except:
                    pass

print('photo sample size:', sample_size)
avg_photo_size = photo_size_bytes / sample_size
total_photo_size_estimate = avg_photo_size * total_num_photo

# get total video size
VIDEO_SIZE = 256 * 2^20 # max Twitter video size is 512MB, so we assume average video size is 256MB
num_total_videos = 0

for file in files:
    data = pd.read_csv(file)
    videos = data['video']
    thumbnails = data['thumbnail']
    photos = data['photos']
    for i, video in enumerate(videos):
        if photos[i] != '[]' and parse_url(photos[i])[0] == thumbnails[i]:
            num_total_videos += video

total_video_size_estimate = VIDEO_SIZE * num_total_videos

total_data_size = total_len_tweets + total_photo_size_estimate + total_video_size_estimate
avg_data_size_per_tweet = total_data_size / total_tweets
avg_data_size_per_tweet = int(avg_data_size_per_tweet)

# avg_data_size_per_tweet = 119159
# photo_sample_size = 279

print(avg_data_size_per_tweet)

In [6]:
# Question 4
# 118118 * num_views = amount of data transfer needed 
# 
total_tweets = 0
data_transfer = 0
for file in files: 
    data = pd.read_csv(file)
    df = pd.DataFrame(data)
    tweets = data['tweet']
    views = len(tweets) * avg_view
    data_amount = views * avg_data_size_per_tweet
    data_transfer += data_amount
    total_tweets += len(tweets)
    

total_data_transfer = data_transfer / total_tweets

print(total_data_transfer)

In [7]:
# Question 5
ELEVEN_YEARS_IN_MIN = 11 * 365 * 24 * 60
avg_views_per_min = total_views / ELEVEN_YEARS_IN_MIN
avg_views_per_min = int(avg_views_per_min)

print(avg_views_per_min)

In [8]:
# Question 6
data_transfer_per_min = int(avg_views_per_min * avg_data_size_per_tweet)
print(data_transfer_per_min)

In [9]:
# Question 7
total_replies = 0
for file in files: 
    data = pd.read_csv(file)
    df = pd.DataFrame(data)
    total_replies += df['nreplies'].sum(axis=0)

print(total_replies/ (11 * 12))
"""
Whenever someone reply to a tweet, the funciton that increments "nreplies" will be executed.

And in total, Elon Musk has 146310 replies in his tweet per month.

Considering that he is 11th most popular person on Twitter in terms of followers,
an average person might get 1/10000 of Elon's replies.
|
That is 15 replies per month, and multiplying that by total user count of 397000000,
we get about 595 million modifications.

Assuming data type of reply count is 4 byte integer,
that is about 2.4 trillion bytes of data modification on a monthly basis
"""